In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/MBX/quip_interface/test


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc","mbx_pbc_no_www_co2ww","mbx_pbc_no_ww_co2w_www_co2ww","mbx_pbc_no_ww_www_co2ww"]

In [9]:
structure_list=["1","2","3"]

In [10]:
dict_version_d_virial=OrderedDict([(struct,
                                    OrderedDict([(version,
                                        OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                       for version in version_list]))
                                    for struct in structure_list])

In [11]:
dict_version_model_virial=OrderedDict([(struct,
                                        OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                       for version in version_list]))
                                       for struct in structure_list]) 

In [12]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""
    pot.reset()

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [13]:
this_root="/home/es732/programs/MBX/quip_interface/test/"

In [14]:
this_struct="1"
this_file="sI_"+this_struct+"_co2_cell2x2x2_rand0_ase.xyz"
this_at_tmp=quippy.Atoms(this_root+this_file)
dict_pot=mbx_functions.create_mbx_potential_variables_co2_h2o(this_at_tmp)

## diagonal_virial=F, no PIPs

In [15]:
this_version=version_list[2]
print this_version

mbx_pbc_no_ww_co2w_www_co2ww


In [16]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [17]:

at= this_at_tmp.copy()

In [18]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-5.05703718e+02  4.02231663e-01  6.70944581e-01]
 [ 4.02231663e-01 -5.06900143e+02 -2.56125534e+00]
 [ 6.70944581e-01 -2.56125534e+00 -5.00884545e+02]]


In [19]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [20]:
for tmp_d in [0.1, 0.01]: #001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F,only 2B co2w PIP

In [21]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_www_co2ww


In [22]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [23]:
at = this_at_tmp.copy()

In [24]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-5.05050749e+02  3.23859134e-01  6.69448353e-01]
 [ 3.23859134e-01 -5.06057609e+02 -2.68173546e+00]
 [ 6.69448353e-01 -2.68173546e+00 -4.98462683e+02]]


In [25]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [26]:
for tmp_d in [0.1,0.01]: #001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [28]:
this_struct="2"
this_file="sI_"+this_struct+"_co2_cell2x2x2_rand0_ase.xyz"
this_at_tmp=quippy.Atoms(this_root+this_file)
dict_pot=mbx_functions.create_mbx_potential_variables_co2_h2o(this_at_tmp)

## diagonal_virial=F, no PIPs

In [29]:
this_version=version_list[2]
print this_version

mbx_pbc_no_ww_co2w_www_co2ww


In [30]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [31]:

at= this_at_tmp.copy()

In [32]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-5.06525512e+02  4.25840561e-01  6.88645189e-01]
 [ 4.25840561e-01 -5.07789064e+02 -2.64209958e+00]
 [ 6.88645189e-01 -2.64209958e+00 -4.98618476e+02]]


In [33]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [34]:
for tmp_d in [0.1, 0.01]: #001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F,only 2B co2w PIP

In [35]:
this_version=version_list[3]
print this_version

mbx_pbc_no_ww_www_co2ww


In [36]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [37]:
at = this_at_tmp.copy()

In [38]:
pot_MBX.reset()
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_struct][this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_struct][this_version]

[[-5.04961208e+02  4.40843460e-01  5.76404836e-01]
 [ 4.40843460e-01 -5.06341899e+02 -2.73745368e+00]
 [ 5.76404836e-01 -2.73745368e+00 -4.93929280e+02]]


In [39]:
for tmp_d in [0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [40]:
for tmp_d in [0.1,0.01]: #001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_struct][this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## Errors in derivative virial / Virial from model (only diagonal)

## only 2B co2w PIP

In [41]:
this_version1="mbx_pbc_no_ww_www_co2ww"
this_version2="mbx_pbc_no_ww_co2w_www_co2ww"
for this_struct in ["1","2"]: #,"2","3"]:
    print this_struct, " CO2"
    for tmp_d in [0.1,0.01,0.001,0.0001,0.00001, 0.000001]:#, 0.0000001, 0.00000001, 0.000000001]:
        print "d =",tmp_d
        v1_numerical = dict_version_d_virial[this_struct][this_version1][tmp_d]
        v2_numerical = dict_version_d_virial[this_struct][this_version2][tmp_d]
        v1_model = dict_version_model_virial[this_struct][this_version1]
        v2_model = dict_version_model_virial[this_struct][this_version2]
        difference_numerical = v1_numerical-v2_numerical
        difference_model=v1_model-v2_model
        print np.diag((difference_numerical -difference_model)/difference_model)

1  CO2
d = 0.1
[ 0.18960841  0.06157223 -0.03383398]
d = 0.01
[ 0.04888325  0.02215187 -0.01497909]
d = 0.001
[ 0.04713314  0.02165877 -0.01540494]
d = 0.0001
[ 0.04711557  0.02165402 -0.01540933]
d = 1e-05
[ 0.04711538  0.02165397 -0.01540937]
d = 1e-06
[ 0.04711535  0.02165402 -0.01540936]
2  CO2
d = 0.1
[0.09817547 0.10374979 0.04844278]
d = 0.01
[ 0.03221112  0.02910284 -0.01592018]
d = 0.001
[ 0.03139323  0.02825523 -0.01704802]
d = 0.0001
[ 0.0313851   0.02824687 -0.01705942]
d = 1e-05
[ 0.03138502  0.02824679 -0.01705953]
d = 1e-06
[ 0.03138494  0.02824682 -0.01705952]
